# Test Your Algorithm

## Instructions
1. From the **Pulse Rate Algorithm** Notebook you can do one of the following:
   - Copy over all the **Code** section to the following Code block.
   - Download as a Python (`.py`) and copy the code to the following Code block.
2. In the bottom right, click the <span style="color:blue">Test Run</span> button. 

### Didn't Pass
If your code didn't pass the test, go back to the previous Concept or to your local setup and continue iterating on your algorithm and try to bring your training error down before testing again.

### Pass
If your code passes the test, complete the following! You **must** include a screenshot of your code and the Test being **Passed**. Here is what the starter filler code looks like when the test is run and should be similar. A passed test will include in the notebook a green outline plus a box with **Test passed:** and in the Results bar at the bottom the progress bar will be at 100% plus a checkmark with **All cells passed**.
![Example](example.png)

1. Take a screenshot of your code passing the test, make sure it is in the format `.png`. If not a `.png` image, you will have to edit the Markdown render the image after Step 3. Here is an example of what the `passed.png` would look like 
2. Upload the screenshot to the same folder or directory as this jupyter notebook.
3. Rename the screenshot to `passed.png` and it should show up below.
![Passed](passed.png)
4. Download this jupyter notebook as a `.pdf` file. 
5. Continue to Part 2 of the Project. 

In [1]:
import glob

import numpy as np
import scipy as sp
import scipy.io
import scipy.signal


def LoadTroikaDataset():
    """
    Retrieve the .mat filenames for the troika dataset.

    Review the README in ./datasets/troika/ to understand the organization of the .mat files.

    Returns:
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that contain reference data
         and  are ordered correspondingly, so that ref_fls[5] is the 
            reference data for data_fls[5], etc...
    """
    data_dir = "./datasets/troika/training_data"
    data_fls = sorted(glob.glob(data_dir + "/DATA_*.mat"))
    ref_fls = sorted(glob.glob(data_dir + "/REF_*.mat"))
    return data_fls, ref_fls

def LoadTroikaDataFile(data_fl):
    """
    Loads and extracts signals from a troika data file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

    Args:
        data_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
    data = sp.io.loadmat(data_fl)['sig']
    return data[2:]


def AggregateErrorMetric(pr_errors, confidence_est):
    """
    Computes an aggregate error metric based on confidence estimates.

    Computes the MAE at 90% availability. 

    Args:
        pr_errors: a numpy array of errors between pulse rate estimates and corresponding 
            reference heart rates.
        confidence_est: a numpy array of confidence estimates for each pulse rate
            error.

    Returns:
        the MAE at 90% availability
    """    
    # Higher confidence means a better estimate. The best 90% of the estimates
    #    are above the 10th percentile confidence.
    
    percentile90_confidence = np.percentile(confidence_est, 10)

    # Find the errors of the best pulse rate estimates
    best_estimates = pr_errors[confidence_est >= percentile90_confidence]
    

    # Return the mean absolute error
    return np.mean(np.abs(best_estimates))

def Evaluate():
    """
    Top-level function evaluation function.

    Runs the pulse rate algorithm on the Troika dataset and returns an aggregate error metric.

    Returns:
        Pulse rate error on the Troika dataset. See AggregateErrorMetric.
    """
    # Retrieve dataset files
    data_fls, ref_fls = LoadTroikaDataset()
    errs, confs = [], []
    for data_fl, ref_fl in zip(data_fls, ref_fls):
        # Run the pulse rate algorithm on each trial in the dataset
        errors, confidence = RunPulseRateAlgorithm(data_fl, ref_fl)
        errs.append(errors)
        confs.append(confidence)
        # Compute aggregate error metric
    errs = np.hstack(errs)
    confs = np.hstack(confs)
    return AggregateErrorMetric(errs, confs)

def RunPulseRateAlgorithm(data_fl, ref_fl):
    """
    Calculates pulse rate estimation and confidence 

    Args:
        data_fl: (str) filepath to a troika .mat file with measurement data.
        ref_fl: (str) filepath to a troika .mat file with reference pulse rate
    Returns:
        errors: numpy array, difference between pulse rate estimates and ground-truth value (reference)
        confidence: numpy array, confidence for each heart rate
    """
    fs = 125
    
    # Load data using LoadTroikaDataFile
    ppg, acc_x, acc_y, acc_z = LoadTroikaDataFile(data_fl)

    # Compute pulse rate estimates and estimation confidence.
    
    # Applying Bandpass filter to ppg and acc signals
    filtered_ppg = BandpassFilter(ppg, fs)
    filtered_acc_x, filtered_acc_y, filtered_acc_z = BandpassFilter(acc_x, fs), BandpassFilter(acc_y, fs), BandpassFilter(acc_z, fs)
    
    acc = np.mean([filtered_acc_x, filtered_acc_y, filtered_acc_z], axis=0)
    
    # Ground truth 
    gt = scipy.io.loadmat(ref_fl)['BPM0'].reshape(-1)
    
    # Make prediction for heart rate, confidence and errors
    preds, confs = make_prediction(acc, ppg, fs)
    
    if len(gt)>len(preds):
        gt = gt[:-1]
    
    errors = np.array(list(np.abs(np.subtract(preds, gt))))
    confidence = np.array(list(confs))
    
    return errors, confidence

def BandpassFilter(signal, fs, pass_band=(40/60.0, 240/60.0)):
    """
    Bandpass filter the signal between 40BPM and 240BPM.
    Args:
        signal: numpy array, for perioduc sugnals
        fs: int, frequency at which a sensor measures a periodic signal
    
    Returns:
    
    filtered signal after appliying Bandpass filter algorythm and return 
    frequences only in 40-240 BPM range.
    """
    b, a = sp.signal.butter(3, (40/60.0, 240/60.0), btype='bandpass', fs=fs)
    return sp.signal.filtfilt(b, a, signal)

def F_transform(sig, fs):
    """
    Fourier Transfrom of the given sequence
    """ 
    freqs = np.fft.rfftfreq(len(sig), 1/fs)
    fft_magnitude = np.abs(np.fft.rfft(sig))
    return (freqs, fft_magnitude)

def conf_fun(freqs, fft, freq):
    """
    Calculate the confidence of transformed data inputs.
    """ 
    window = (freqs > freq - 1) & (freqs < freq + 1)  
    confidence = np.sum(fft[window]) / np.sum(fft)
    return confidence

def make_prediction(acc, ppg, fs, window_len_s = 8, window_shift_s = 2):
    """
    Predict heart rate and calculate confidence using ppg and accelerometer data
    """
    
    min_bandpass = 40/60.0
    max_bandpass = 240/60.0
     
    window_length = window_len_s * fs
    window_shift = window_shift_s * fs
    
    pred = []
    confidences = []
     
    filtered_signals = {'acc':acc, 'ppg':ppg}
    
    #indices of the startig points
    indices = np.arange(start=0, stop=len(ppg) - window_length, step=window_shift)
    
    # Calculations with shifting windows
    for i in indices:
        
        window, freqs, freq, fft = {}, {}, {}, {}
        # For each signal we calculate the estimation for each window, apply Fourier transformation and filter out frequencies
        for name, s in filtered_signals.items():
            
            window[name] = s[i:i+window_length]
            freqs[name], fft[name] = F_transform(window[name], fs)
            fft[name][(freqs[name] <= min_bandpass) | (freqs[name] >= max_bandpass)] = 0.0
#           Pick frequency with the largerst magnitude
            freq[name] = freqs[name][np.argmax(fft[name], axis=0)] 
        
        ppg_freq, ppg_fft, ppg_freqs = freq['ppg'], fft['ppg'], freqs['ppg']
        acc_freq = freq['acc']
        
        freq = ppg_freq
        conf = conf_fun(ppg_freqs, ppg_fft, ppg_freq)
        
        # New estimation for second strongest frequency.
        if abs(ppg_freq - acc_freq) == 0:
            
            # Frequency with the second largerst FFT
            sec_freq = ppg_freqs[np.argsort(ppg_fft, axis=0)[-2]]
            sec_conf = conf_fun(ppg_freqs, ppg_fft, sec_freq)
            
            # Pick the larger freq
            if sec_conf > conf:
                freq, conf = sec_freq, sec_conf
        
        pred.append(freq * 60)
        confidences.append(conf)
    
     
    return pred, confidences
